In [1]:
import pandas as pd

In [2]:
# Import data from excel, switch to sqlite later on
#INF_DF = pd.read_excel (r'/Users/JZ/Downloads/TEST_INFO.xlsx')
#FINANCIALS_DF = pd.read_excel (r'/Users/JZ/Downloads/FIN_ALL.xlsx')
# HIST_PRICE_DF = pd.read_excel (r'/Users/JZ/Downloads/PRICES_ALL.xlsx')

TIC_FLAT = pd.read_excel(r'/Users/joezhou/Downloads/TIC_FLAT.xlsx')



In [9]:


HIST_PRICE_DFs = HIST_PRICE_DF[HIST_PRICE_DF['TickName'].isin(['CBA.AX','ANZ.AX','APT.AX','ZIP.AX'])]

In [ ]:
import time
start_time = time.time()


INF_DF = pd.DataFrame()
CALC_FEATURES_DF = pd.DataFrame()

FIN_DF = pd.DataFrame()
BS_DF = pd.DataFrame()
CF_DF = pd.DataFrame()

def Info_Extract(TickName):
    
    global INF_DF
    global CALC_FEATURES_DF
    global FIN_DF
    global BS_DF
    global CF_DF
    
    try:
        tick = yf.Ticker(TickName)
        
        INF = tick.info
        FIN_OG = tick.financials
        FIN = tick.financials.transpose()
        BS_OG = tick.balance_sheet
        BS = tick.balance_sheet.transpose()
        CF = tick.cashflow
        DIV = tick.dividends        
        
     #----------------------------------------------------------------------------------------------------------
        #append full data
        
        INF_DF = INF_DF.append(INF, ignore_index=True) 
        
        FIN_OG2 = FIN_OG.reset_index()
        FIN_OG2.rename(columns = {'index':'Metric'}, inplace = True)
        FIN_OG2['TickName']=TickName
        FIN_DF = FIN_DF.append(FIN_OG2, ignore_index=True)
        
        BS_OG2 = BS_OG.reset_index()
        BS_OG2.rename(columns = {'index':'Metric'}, inplace = True)
        BS_OG2['TickName']=TickName
        BS_DF = BS_DF.append(BS_OG2, ignore_index=True)
                
        CF2 = CF.reset_index()
        CF2.rename(columns = {'index':'Metric'}, inplace = True)
        CF2['TickName']=TickName
        CF_DF = CF_DF.append(CF2, ignore_index=True)      
        

   #----------------------------------------------------------------------------------------------------------
        #Part 0: Calculate WACC return
        
        # Calculate TAX rate
        
        try:
            FIN['ITE']=FIN['Income Tax Expense']
        except KeyError:
            FIN['ITE']=0
            
        try:
            FIN['IBT']=FIN['Income Before Tax']
        except KeyError:
            FIN['IBT']=0
        
       
        TAX_RATE = (FIN['ITE']/FIN['IBT']).median()
    
        
        try:
            BS['ld']=BS['Long Term Debt']
        except KeyError:
            BS['ld']=0
            
        try:
            BS['sd']=BS['Short Long Term Debt']
        except KeyError:
            BS['sd']=0         
  
        try:
            BS['eq']=BS['Total Stockholder Equity']
        except KeyError:
            BS['eq']=0  
            
        try:
            BS['ta']=BS['Total Assets']
        except KeyError:
            BS['ta']=0  
            
        
        BS['CALC Capital'] = BS['sd']+BS['ld']+BS['eq']
        debt_p = (BS['ld']+BS['sd'])/BS['CALC Capital']
        DEBT_PERC_CAPITAL = debt_p.iloc[[0]].max()
        EQUITY_PERC_CAPITAL = 1-DEBT_PERC_CAPITAL
     
        
        #calculate cost of debt
        
        debt_val = BS['ld']/BS['ta']
        DEBT_COST = debt_val.median()

        
        #calculate equity rate using 2 different approach

        #Method 1 use the CAPM model
        
        
        if INF['dividendYield'] is not None:
            INF['DY']=INF['dividendYield']
        else:
            INF['DY']=0  
            
        if INF['beta'] is not None:
            INF['be']=INF['beta']
        else:
            INF['be']=0 
        
     
        rf_rtn = .025
        EQUITY_COST_CAPM = rf_rtn + ((INF["DY"]-rf_rtn)*INF["be"])
    
        
            
        #Method 2 use dividend approach
        if INF['dividendRate'] is not None:
            INF['dr']=INF['dividendRate']
        else:
            INF['dr']=0
    
            
        if INF['previousClose'] is not None:
            INF['pc']=INF['previousClose']
        else:
            INF['pc']=0
            
        div_growth = 0
    
        EQUITY_COST_DIV = (INF["dr"]/INF["pc"])+div_growth
            
        #Calculate minimum
        EQUITY_COST_AVG = min(EQUITY_COST_CAPM,EQUITY_COST_DIV)       

        WACC_CAPM =(EQUITY_PERC_CAPITAL*EQUITY_COST_CAPM)+(DEBT_PERC_CAPITAL*DEBT_COST)*(1-TAX_RATE)       
        
        WACC_DIV =(EQUITY_PERC_CAPITAL*EQUITY_COST_DIV)+(DEBT_PERC_CAPITAL*DEBT_COST)*(1-TAX_RATE)  
        
        WACC_AVG =(EQUITY_PERC_CAPITAL*EQUITY_COST_AVG)+(DEBT_PERC_CAPITAL*DEBT_COST)*(1-TAX_RATE)  
        
        
    #----------------------------------------------------------------------------------------------------------
         
        #Part 1: Dividend
        try:          
            DIV_PA = DIV.resample('Y').sum()
            DIV_L4_AVG = DIV_PA.iloc[[-5,-4,-3,-2]].sum()/4
            DIV_L4_STD = DIV_PA.iloc[[-5,-4,-3,-2]].std()
            DIVIDEND_BASE = DIV_L4_AVG - DIV_L4_STD   
        except IndexError:
            DIV_L4_AVG = 0
            DIV_L4_STD = 0
            DIVIDEND_BASE = 0    
            
        #Calculate NPV dividends
        
        DIVIDEND_BASE_NPV = (DIVIDEND_BASE/(1+WACC_AVG)**1) + (DIVIDEND_BASE/(1+WACC_AVG)**2) + (DIVIDEND_BASE/(1+WACC_AVG)**3)
        
        
        
      
    #----------------------------------------------------------------------------------------------------------
        
        YSTDAY_PRICE = INF['previousClose']
        
            
            
        #Part 2: Company Growth
        #Quarterly earling growth
        
        
        if INF['earningsQuarterlyGrowth'] is not None:
            INF['eqg']=INF['earningsQuarterlyGrowth']
        else:
            INF['eqg']=0  
            
        REV_GWTH_PA = INF['eqg']*4/100   
                

            
        #Part 4: Valuation
        #Net tangible assets
        try:
            VAL_NTA = BS['Net Tangible Assets'].iloc[[0]].max()/INF['floatShares']
        except IndexError:
            VAL_NTA = 0
        
        #PE ratio
        
        if INF['enterpriseValue'] is not None:
            INF['IV']=INF['enterpriseValue']
        else:
            INF['IV']=0  
        
        if INF['enterpriseToRevenue'] is not None:
            INF['ETR']=INF['enterpriseToRevenue']
        else:
            INF['ETR']=0         
        
   
        if INF['profitMargins'] is not None:
            INF['PM']=INF['profitMargins']
        else:
            INF['PM']=0  
            
        if INF['sharesOutstanding'] is not None:
            INF['SO']=INF['sharesOutstanding']
        else:
            INF['SO']=0 
            
#         if INF['trailingPE'] is not None:
#             INF['TPE']=float(INF['trailingPE'])
#         else:
#             INF['TPE']=0 
    
        try:
            INF['TPE']=float(INF['trailingPE'])
        except KeyError:
            INF['TPE']=0
            
 
            
        try:
            VAL_PE_RATIO = (INF['IV']/INF['ETR']) * INF['PM']/INF['SO']*INF['TPE']
        except ZeroDivisionError:
            VAL_PE_RATIO = 0
        
        REV_YOY = FIN['Total Revenue'].iloc[[0]].max()/ FIN['Total Revenue'].iloc[[1]].max() -1
        REVENUE_AVG=FIN['Total Revenue'].mean()
        TAX_AVG =FIN['Income Tax Expense'].mean()
        OPEX_AVG=FIN['Total Operating Expenses'].mean()
        NET_INCOME = REVENUE_AVG - TAX_AVG - OPEX_AVG
        OPEX_REV_RATE = OPEX_AVG / REVENUE_AVG        
            
            
 #----------------------------------------------------------------------------------------------------------          
       #NPV Cashflows 
        FIN_OG = tick.financials
        NET_INCOME_BASE = FIN_OG[(FIN_OG.index == "Net Income Applicable To Common Shares")].median().mean()
        NPV_NET_INCOME = (NET_INCOME_BASE/(1+WACC_AVG)**1) + (NET_INCOME_BASE/(1+WACC_AVG)**2) + (NET_INCOME_BASE/(1+WACC_AVG)**3)
        
        NPV_NET_INCOME_PERP = NET_INCOME_BASE / WACC_AVG /INF['sharesOutstanding'] 
        NPV_DIVIDEND_BASE_PERP = DIVIDEND_BASE / WACC_AVG
      
        POSSIBLE_GAIN_REV = NPV_NET_INCOME_PERP - YSTDAY_PRICE
    
        #Append features
        
        x_list = [TickName,TAX_RATE,YSTDAY_PRICE,DIV_L4_AVG,DIV_L4_STD,DIVIDEND_BASE,REV_GWTH_PA,VAL_NTA,VAL_PE_RATIO,
                 DEBT_PERC_CAPITAL,EQUITY_PERC_CAPITAL,DEBT_COST,
                  EQUITY_COST_CAPM,EQUITY_COST_DIV,EQUITY_COST_AVG,WACC_CAPM,WACC_DIV,WACC_AVG,DIVIDEND_BASE_NPV,
                  REV_YOY,REVENUE_AVG,TAX_AVG,OPEX_AVG,NET_INCOME,OPEX_REV_RATE,
                  NET_INCOME_BASE,NPV_NET_INCOME,NPV_NET_INCOME_PERP,NPV_DIVIDEND_BASE_PERP,POSSIBLE_GAIN_REV
                 ]
        #,DEBT_COST,EQUITY_COST_CAPM,EQUITY_COST_DIV,EQUITY_COST_AVG]
        x_list_df = pd.DataFrame(x_list).transpose()
        CALC_FEATURES_DF = CALC_FEATURES_DF.append(x_list_df, ignore_index=True)          
                


        print("finished",TickName)
        
    except Exception:
        print("Error with",TickName)
        pass

    return()


for company in ASX_TICKER_LIST:
    Info_Extract(company)



print("--- %s seconds ---" % (time.time() - start_time))



In [ ]:
        try:
            FIN['ITE']=FIN['Income Tax Expense']
        except KeyError:
            FIN['ITE']=0
            
        try:
            FIN['IBT']=FIN['Income Before Tax']
        except KeyError:
            FIN['IBT']=0
        
       
        TAX_RATE = (FIN['ITE']/FIN['IBT']).median()
    
        
        try:
            BS['ld']=BS['Long Term Debt']
        except KeyError:
            BS['ld']=0
            
        try:
            BS['sd']=BS['Short Long Term Debt']
        except KeyError:
            BS['sd']=0         
  
        try:
            BS['eq']=BS['Total Stockholder Equity']
        except KeyError:
            BS['eq']=0  
            
        try:
            BS['ta']=BS['Total Assets']
        except KeyError:
            BS['ta']=0  
            
        
        BS['CALC Capital'] = BS['sd']+BS['ld']+BS['eq']
        debt_p = (BS['ld']+BS['sd'])/BS['CALC Capital']
        DEBT_PERC_CAPITAL = debt_p.iloc[[0]].max()
        EQUITY_PERC_CAPITAL = 1-DEBT_PERC_CAPITAL
     
        
        #calculate cost of debt
        
        debt_val = BS['ld']/BS['ta']
        DEBT_COST = debt_val.median()

        
        #calculate equity rate using 2 different approach

        #Method 1 use the CAPM model
        
        
        if INF['dividendYield'] is not None:
            INF['DY']=INF['dividendYield']
        else:
            INF['DY']=0  
            
        if INF['beta'] is not None:
            INF['be']=INF['beta']
        else:
            INF['be']=0 
        
     
        rf_rtn = .025
        EQUITY_COST_CAPM = rf_rtn + ((INF["DY"]-rf_rtn)*INF["be"])
    
        
            
        #Method 2 use dividend approach
        if INF['dividendRate'] is not None:
            INF['dr']=INF['dividendRate']
        else:
            INF['dr']=0
    
            
        if INF['previousClose'] is not None:
            INF['pc']=INF['previousClose']
        else:
            INF['pc']=0
            
        div_growth = 0
    
        EQUITY_COST_DIV = (INF["dr"]/INF["pc"])+div_growth
            
        #Calculate minimum
        EQUITY_COST_AVG = min(EQUITY_COST_CAPM,EQUITY_COST_DIV)       

        WACC_CAPM =(EQUITY_PERC_CAPITAL*EQUITY_COST_CAPM)+(DEBT_PERC_CAPITAL*DEBT_COST)*(1-TAX_RATE)       
        
        WACC_DIV =(EQUITY_PERC_CAPITAL*EQUITY_COST_DIV)+(DEBT_PERC_CAPITAL*DEBT_COST)*(1-TAX_RATE)  
        
        WACC_AVG =(EQUITY_PERC_CAPITAL*EQUITY_COST_AVG)+(DEBT_PERC_CAPITAL*DEBT_COST)*(1-TAX_RATE)  
        
        
    #----------------------------------------------------------------------------------------------------------
         
        #Part 1: Dividend
        try:          
            DIV_PA = DIV.resample('Y').sum()
            DIV_L4_AVG = DIV_PA.iloc[[-5,-4,-3,-2]].sum()/4
            DIV_L4_STD = DIV_PA.iloc[[-5,-4,-3,-2]].std()
            DIVIDEND_BASE = DIV_L4_AVG - DIV_L4_STD   
        except IndexError:
            DIV_L4_AVG = 0
            DIV_L4_STD = 0
            DIVIDEND_BASE = 0    
            
        #Calculate NPV dividends
        
        DIVIDEND_BASE_NPV = (DIVIDEND_BASE/(1+WACC_AVG)**1) + (DIVIDEND_BASE/(1+WACC_AVG)**2) + (DIVIDEND_BASE/(1+WACC_AVG)**3)
        
        
        
      
    #----------------------------------------------------------------------------------------------------------
        
        YSTDAY_PRICE = INF['previousClose']
        
            
            
        #Part 2: Company Growth
        #Quarterly earling growth
        
        
        if INF['earningsQuarterlyGrowth'] is not None:
            INF['eqg']=INF['earningsQuarterlyGrowth']
        else:
            INF['eqg']=0  
            
        REV_GWTH_PA = INF['eqg']*4/100   
                

            
        #Part 4: Valuation
        #Net tangible assets
        try:
            VAL_NTA = BS['Net Tangible Assets'].iloc[[0]].max()/INF['floatShares']
        except IndexError:
            VAL_NTA = 0
        
        #PE ratio
        
        if INF['enterpriseValue'] is not None:
            INF['IV']=INF['enterpriseValue']
        else:
            INF['IV']=0  
        
        if INF['enterpriseToRevenue'] is not None:
            INF['ETR']=INF['enterpriseToRevenue']
        else:
            INF['ETR']=0         
        
   
        if INF['profitMargins'] is not None:
            INF['PM']=INF['profitMargins']
        else:
            INF['PM']=0  
            
        if INF['sharesOutstanding'] is not None:
            INF['SO']=INF['sharesOutstanding']
        else:
            INF['SO']=0 
            
#         if INF['trailingPE'] is not None:
#             INF['TPE']=float(INF['trailingPE'])
#         else:
#             INF['TPE']=0 
        try:
            INF['TPE']=float(INF['trailingPE'])
        except KeyError:
            INF['TPE']=0
 
            
        try:
            VAL_PE_RATIO = (INF['IV']/INF['ETR']) * INF['PM']/INF['SO']*INF['TPE']
        except ZeroDivisionError:
            VAL_PE_RATIO = 0
        
        REV_YOY = FIN['Total Revenue'].iloc[[0]].max()/ FIN['Total Revenue'].iloc[[1]].max() -1
        REVENUE_AVG=FIN['Total Revenue'].mean()
        TAX_AVG =FIN['Income Tax Expense'].mean()
        OPEX_AVG=FIN['Total Operating Expenses'].mean()
        NET_INCOME = REVENUE_AVG - TAX_AVG - OPEX_AVG
        OPEX_REV_RATE = OPEX_AVG / REVENUE_AVG        
            
            
 #----------------------------------------------------------------------------------------------------------          
       #NPV Cashflows 
        FIN_OG = tick.financials
        NET_INCOME_BASE = FIN_OG[(FIN_OG.index == "Net Income Applicable To Common Shares")].median().mean()
        NPV_NET_INCOME = (NET_INCOME_BASE/(1+WACC_AVG)**1) + (NET_INCOME_BASE/(1+WACC_AVG)**2) + (NET_INCOME_BASE/(1+WACC_AVG)**3)
        
        NPV_NET_INCOME_PERP = NET_INCOME_BASE / WACC_AVG /INF['sharesOutstanding'] 
        NPV_DIVIDEND_BASE_PERP = DIVIDEND_BASE / WACC_AVG
      
        POSSIBLE_GAIN_REV = NPV_NET_INCOME_PERP - YSTDAY_PRICE